In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt
import pathlib
import os

AUTOTUNE = tf.data.experimental.AUTOTUNE
tf.keras.backend.clear_session()  # For easy reset of notebook state.

tf.__version__
tf.executing_eagerly()


True

In [7]:

ds = tfds.load(name="imagenette", with_info=True, split=["train[:1000]","validation[:100]"])
ds_train=ds[0][0]
ds_test=ds[0][1]
assert isinstance(ds_train, tf.data.Dataset)

def normalize(x):
    y = {'image': tf.image.convert_image_dtype(x['image'], tf.float32), 'label': x['label']}
    y = (tf.image.resize(y['image'], (224,224)), y['label'])
    return y
    
ds_train = ds_train.map(lambda x: normalize(x))
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(100)
ds_train = ds_train.batch(4)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(4)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)


In [ ]:
class CustomModel(tf.keras.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.layer1 = layers.Dense(1024, activation='relu')
        self.layer2 = layers.Dense(128, activation='relu')
        self.layer3 = layers.Dense(10, activation='softmax')
    def call(self,inputs):
        #print(inputs)
        x = layers.Flatten()(inputs)
        #print(x)
        x = self.layer1(x)
        x = self.layer2(x)
        return self.layer3(x)

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(224, 224, 3)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

model = CustomModel()

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

model.fit(
    x=ds_train,
    #steps_per_epoch=1,
    epochs=15,
    validation_data=ds_test,
)

Epoch 1/15
250/250 [==============================] - 171s 683ms/step - loss: 37.1163 - accuracy: 0.1590 - val_loss: 2.2970 - val_accuracy: 0.0700
Epoch 2/15
250/250 [==============================] - 156s 625ms/step - loss: 2.3602 - accuracy: 0.1180 - val_loss: 2.2976 - val_accuracy: 0.0800
Epoch 3/15
250/250 [==============================] - 158s 633ms/step - loss: 2.3068 - accuracy: 0.1150 - val_loss: 2.3008 - val_accuracy: 0.0700
Epoch 4/15
211/250 [========================>.....] - ETA: 23s - loss: 2.3008 - accuracy: 0.1126

In [ ]:
for example in ds_train.take(1):  # Only take a single example
    print(example['image'].shape)
    image, label = example["image"], example["label"]
    plt.imshow(image.numpy()[:, :, 0].astype(np.float32), cmap=plt.get_cmap("gray"))
    print("Label: %d" % label.numpy())

In [ ]:
ds_train = ds_train.repeat().shuffle(1024).batch(32)

# prefetch will enable the input pipeline to asynchronously fetch batches while
# your model is training.
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
tfds.show_examples(ds_train[1] ,ds_train[0]['train'])